---
# Projectr : 하루시작 지하철 혼잡도 분석

### Description : 
- <a><span style = "color: #FFBE98">
    <b>BERT Model을 활용한 오늘의 주요 키워드 및 뉴스 추천하기</b>
  </a>

### Author : Zen Den

### Date : 2024. 06. 13. (Thu) ~

### Detail :
  - Crawling 평균 소요시간 <br><br>
    - 언론사별 랭킹뉴스의 언론사명, Title, Link: 1분 50초 <br><br>
    - 뉴스 기사의 본문: 15분 30초 <br><br>

  > (1) 환경 설정 및 라이브러리 설치
  - 웹 크롤링을 위한 requests와 BeautifulSoup, Selenium <br>
  - 데이터 처리를 위한 pandas <br>
  - 텍스트 처리를 위한 nltk <br>
  - 딥러닝 모델인 BERT를 활용하기 위한 transformers와 torch <br>
  - 자연어 처리 위한 sklearn

  > (2) Web Crawling
  - 네이버 뉴스 페이지를 요청하고, BeautifulSoup으로 HTML 파싱 <br>
  - 각 언론사별로 상위 5개 뉴스 기사 제목 및 Link 추출

  > (3) Keyword 추출
  - 각 뉴스 기사 본문을 가져와서 빈도수 높은 단어 추출 <br>
  - 불용어 제거 및 형태소 분석을 통해 주요 Keyword 도출

  > (4) 뉴스 추천 시스템 구축
  - BERT Model을 활용하여 각 뉴스 기사 Embedding 생성 <br>
  - 코사인 유사도를 사용하여 주요 Keyword와 유사한 뉴스 기사 추천

### Update: 
- 2024.06.13. (Thu) K.Zen : <br><br>
  - <span style = "color: cyan">
      <b>Deep Learning(BERT Model)</b>
      을 활용한 뉴스 추천하기
    </span>

---
# Environment Setting and Import Library Package

## Basic

In [1]:
import pandas as pd, numpy as np

## File System

In [2]:
import os; from datetime import datetime

## Crawling

In [3]:
import requests # 인터넷에서 Data를 가져오기 위한 Library (웹페이지에 접속하고 HTML 코드를 가져오기 위해 사용)
from bs4 import BeautifulSoup # 웹 페이지 내용을 분석하기 위한 Library (가져온 HTML 코드에서 우리가 필요한 정보를 추출하기 위해 사용)

import time # 대기 시간을 추가하기 위한 Library (요청 사이에 랜덤한 시간을 기다리기 위해 사용)
import random # Random한 대기 시간을 만들기 위한 Library
from tqdm import tqdm # Crawling 진행 상황을 체크하기 위한 Module (진행 상황을 시각적으로 보여주기 위해 사용)

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

## Bag of Words (BoW)

In [4]:
import nltk # Natural Language Toolkit (자연어 처리를 위해 사용)
from konlpy.tag import Okt
from collections import Counter # 단어의 빈도를 계산하기 위해 사용

# 1. nltk Data Download
nltk.download('punkt')

# 2. 한국어 불용어 사전
# ****************************************************************
## 한국어 불용어 모음집 불러오기
stopword_list = pd.read_csv("Data/stopword.txt", header = None)
# ****************************************************************
stopword_list[0] = stopword_list[0].apply(lambda x: x.strip())
stopwords = stopword_list[0].to_numpy()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gwangyeong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Deep Learning - BERT

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer
import torch

# Web Crawling

In [6]:
# 1. 뉴스 Crawling
def get_news_links_by_press (url) :
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
  response = requests.get(url, headers = headers) # url(페이지)에 접속
  soup = BeautifulSoup(response.content, 'html.parser') # HTML 코드를 파싱(분석)하여 soup 객체에 저장
  
  press_data = {}
  press_sections = soup.select('.rankingnews_box')
  
  for press_section in tqdm(press_sections, desc = "언론사별 뉴스 Crawling") :
    press_name = press_section.select_one('.rankingnews_name').get_text(strip = True)
    news_links = set()  # 중복 제거를 위한 set 사용
    for item in press_section.select('li a') :
      title = item.get_text(strip = True)
      link = item['href']
      if title and link and "동영상" not in title :  # Title이 존재하고 "동영상"이 포함되지 않은 경우에만 추가
        news_links.add((title, link))
    press_data[press_name] = list(news_links)[:5]  # 다시 list로 변환 후 상위 5개만 저장
    
    # *********************************************
    # 각 언론사별 뉴스 Crawling 후 대기 시간 추가
    time.sleep(random.uniform(0.5, 2.0))
    # *********************************************
  
  return press_data

In [7]:
base_url = 'https://news.naver.com/main/ranking/popularDay.naver'
press_news_data = get_news_links_by_press(base_url)

# 1. 뉴스 DataFrame 생성
news_list = []
for press_name, news_data in press_news_data.items() :
    for title, link in news_data :
        news_list.append([press_name, title, link])
news_df = pd.DataFrame(news_list, columns = ['Press', 'Title', 'Link'])

news_df.head()

언론사별 뉴스 Crawling: 100%|██████████| 82/82 [01:44<00:00,  1.27s/it]


,Press,Title,Link
0,조선일보,"비비탄총 든 美10대에 총격 사망, 일반인이 매장 지킨다며 쐈다",https://n.news.naver.com/article/023/000384004...
1,조선일보,"강남대로 횡단보도 건너던 50대 여성, 신호 위반 버스에 치여 숨져",https://n.news.naver.com/article/023/000384009...
2,조선일보,문신男 2000명 웃통 벗겨 끌고 갔다…증거 없어도 체포하는 이 나라,https://n.news.naver.com/article/023/000384006...
3,조선일보,병원 노조 “의사들 병원 지켜라… 무기한 휴진으로 파생된 업무 거부”,https://n.news.naver.com/article/023/000384012...
4,조선일보,“부 대물림 안해” 515억 카이스트 기부 정문술 전 미래산업 회장 별세,https://n.news.naver.com/article/023/000384007...


# Keyword 추출

In [8]:
# Chrome Browser와 Chrome Driver Version 확인 및 WebDriver 객체 생성
chrome_options = webdriver.ChromeOptions()
# ******************************************************
chrome_options.add_argument('headless') # Run chrome browser in the background
chrome_options.add_argument('window-size = 1920x1080')  # Chrome Browser Window Size
# ******************************************************
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)

# 1. 뉴스 기사 본문 가져오기 함수 정의
def get_article_text (url) :
    driver.get(url)

    # **************************************************************************
    # 기본 Path 설정
    today = datetime.now().strftime('%Y%m%d')   # 오늘 날짜 가져오기
    base_path = f"Data/ScreenShot/By_Press/{today}"

    ## By_Press 폴더 생성
    os.makedirs(base_path, exist_ok = True)

    # 동일한 By_Press/today Folder가 존재하는지 확인하고, 존재한다면 하위에 새로운 Folder 생성
    new_path = base_path
    while os.path.exists(new_path) :
        new_path = os.path.join(base_path, str(len(os.listdir(base_path)) + 1)) # index는 폴더 내 파일 개수 + 1로 설정
    
    # 새로운 Folder 생성
    os.makedirs(new_path, exist_ok = True)

    # 기본 File Name 설정
    base_filename = news_df[news_df['Link'] == url]['Title'].values[0]
    extension = ".png"
    name_index = 1  # File Name에 추가될 숫자
    new_filename = base_filename + extension

    # File Path 생성
    screenshot_name = os.path.join(new_path, new_filename)

    # 동일한 File Name이 존재하는지 확인하고, 존재한다면 새로운 File Name 생성
    while os.path.exists(screenshot_name) :
        new_filename = f"{base_filename}_{name_index}{extension}"
        screenshot_name = os.path.join(new_path, new_filename)
        name_index += 1
    
    # Browser 최대화
    driver.maximize_window()
    
    # 현재 화면 Capture하기
    driver.save_screenshot(screenshot_name)
    # **************************************************************************

    html = driver.page_source
    article_soup = BeautifulSoup(html, "html.parser")
    content = article_soup.select_one("#contents")
    if content :
        # 공백과 HTML Tag 제거
        text =''.join(content.text.split())
        # *******************************************
        # 요청 후 임의의 시간만큼 대기 (Page Loaded)
        time.sleep(random.uniform(0.5, 2.0))
        # *******************************************
        
        return text

# 2. 뉴스 기사 본문 수집
content_pbar = tqdm(news_df['Link'], desc = "뉴스 기사 본문 Crawling 진행 중", unit = "Link")
news_df['content'] = [get_article_text(url) for url in content_pbar]

# Browser 종료 (모든 Tab 종료)
driver.quit()

# 3. 키워드 추출 함수 정의
def get_keywords (text, num_keywords = 10) :
    okt = Okt()
    tokens = okt.nouns(text)
    tokens = [word for word in tokens if word not in stopwords and len(word) > 1]
    return [word for word, freq in Counter(tokens).most_common(num_keywords)]

# 4. 각 언론사별 키워드 추출
text_pbar = tqdm(news_df['content'], desc = "Text 처리 중", unit = "기사")
news_df['keywords'] = [get_keywords(text) for text in text_pbar]
news_df['keywords'].head()

print(news_df['content'].head().values)
print('🍋' * 80)
print(news_df['keywords'].head().values)

Text 처리 중: 100%|██████████| 406/406 [02:02<00:00,  3.32기사/s]

['지난5일(현지시각)미국워싱턴주렌턴의한스포츠용품매장주차장에서\'가짜\'경비원이10대소년을강도로착각해총으로쏴살해하는사건이발생했다./RentonPoliceDepartment미국에서비비탄총을든10대를강도로오인해‘가짜’경비원이총으로쏴살해하는사건이발생했다.최근미국전역에서집이나회사에접근했다는이유만으로무고한이들에게총격을가하는사건이잇따라발생하면서이사건은더욱논란이되고있다.12일(현지시각)미국CNN,ABC등에따르면미국워싱턴주킹카운티검찰은살인혐의로아론마이어스(51)를기소했다.검찰의기소장에따르면,사건은지난5일저녁워싱턴주렌턴에있는‘빅5′스포츠용품매장주차장에서벌어졌다.17세소년하즈라트알리로하니와친구2명은고장난비비탄총을교환하려고이매장에들어서고있었다.주차장에서이모습을본마이어스는‘글록권총’을가진세명의10대가강도행각을저지를것이라고생각했다.즉시차량에서내려권총을겨눈마이어스는소년들에게‘총을버리고땅에엎드리라’고명령했다.마이어스는아무도자기말을듣지않았고,로하니가허리춤에손을뻗고있었다고주장했다.총을꺼내자신을죽일것이라고생각한마이어스는로하니를향해최소7발의총격을가했다.로하니는옆구리와등에6발을맞고현장에서사망했다.하지만,경찰이확보한CCTV영상에는마이어스의주장과모순되는장면이담겨있었다.마이어스의명령에10대들은가짜총을땅바닥에내려놓았다.로하니의친구는“그저비비탄총일뿐”이라고여러차례말했다고진술했다.그럼에도마이어스는로하니에게태클을걸어그를땅바닥에엎드리게했다.이후마이어스는소년에게총격을가했다.더큰문제는마이어스가해당매장에서고용한정식경비원이아니라는점이었다.경비회사를운영중인마이어스는자신의13세아들이빅5매장옆에서주짓수수업을듣는동안주차장에서‘오버워치’를하고있었다고한다.오버워치는특정장소를감시하는행위를뜻하는군대용어다.마이어스는“과거에주차장에서수많은범죄가발생하는것을목격했기때문에그장소에서감시하고있었다”며“무고한사람을해치는것을막고아들을보호하기위해내가나서야할의무가있었다”고주장했다.마이어스의변호사는“그와가족은한소년이목숨을잃었다는사실에큰충격을받았다”면서도“문제의저녁,마이어스는진심으로무장강도를목격했다고믿었고자신을방어하기위해무기를사용했다”고했다.마이어스는2

# 뉴스 추천 시스템 구축

In [9]:
# 1. BERT 모델 및 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

"""
    with torch.no_grad():
    - 블록을 사용하여 모델의 출력을 계산할 때 그래디언트를 저장하지 않도록 했습니다.
        - 이는 메모리 사용량을 줄이고 성능을 향상시킵니다.
"""
# 2. 문장 임베딩 생성 함수 정의
def get_bert_embedding (text) :
    inputs = tokenizer(
        text,
        return_tensors = 'pt',
        truncation = True,
        padding = True,
        max_length = 512
    )
    with torch.no_grad() :
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim = 1).detach().numpy()

# 3. 각 뉴스 기사 임베딩 생성
embed_pbar = tqdm(news_df['content'], desc = "Embedding 처리 중", unit = "text")
news_df['embedding'] = [get_bert_embedding(text) for text in embed_pbar]


"""
    keyword_text 생성 시, sum(news_df['keywords'], [])을 사용하여 List의 List를 평탄화한 후 Keyword를 결합합니다.
"""
# 4. 주요 Keyword 임베딩 생성
keyword_text = ' '.join(sum(news_df['keywords'], []))
keyword_embedding = get_bert_embedding(keyword_text)


"""
    np.vstack(df['embedding'].values:
    - numpy 배열로 변환하는 부분입니다.
        - 이는 embeddings 변수를 numpy 배열로 변환하여 코사인 유사도를 계산할 수 있게 합니다.
"""
# 5. 코사인 유사도 계산 및 뉴스 추천
def recommend_news (df, keyword_embedding, top_n = 10) :
    embeddings = np.vstack(df['embedding'].values)
    similarities = cosine_similarity(embeddings, keyword_embedding.reshape(1, -1)).flatten()
    df['similarity'] = similarities
    return df.nlargest(top_n, 'similarity')

recommended_news = recommend_news(news_df, keyword_embedding)
recommended_news[['Press', 'Title', 'similarity']]

Embedding 처리 중: 100%|██████████| 406/406 [02:18<00:00,  2.92text/s]


,Press,Title,similarity
254,경기일보,수원의 한 도로서 차량 인도 돌진…운전자 등 4명 부상,0.860929
253,경기일보,"정신병원서 지적장애인 폭행한 요양보호사, 검찰 송치",0.860321
283,전주MBC,군산 태양광 비리 의혹..신영대 의원 전 보좌관 '구속',0.854462
314,강원도민일보,춘천 삼악산 케이블카 전망대 인근서 60대 추정 남성 숨진 채 발견,0.853928
70,MBN,[단독] 10대 남성 호텔에서 숨진 채 발견..경찰 조사 착수,0.853112
392,JIBS,호텔 주차장 2층서 60대 여성 몰던 제네시스 SUV 추락,0.851154
13,YTN,해병대 부사관이 군 숙소에서 마약 투약...긴급체포,0.849962
33,연합뉴스,"BBQ, 땡초숯불양념치킨 출시…""매운맛 찾는 MZ세대 겨냥""",0.848499
326,한경비즈니스,"[속보]의협 ""주말까지 정부 입장 변화 없으면 '집단 휴진' 강행할 것""",0.848396
198,강원일보,삼악산 호수케이블카 전망대 인근서 60대 숨진 채 발견,0.848379
